In [ ]:
# default_exp graph_functions

# Graph Functions

> Operations on networkx MultiDiGraph objects, including assigning and obtaining attributes from nodes and edges, converting edges into nodes, and sorting and labeling graphs. 

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import warnings
with warnings.catch_warnings(): #ignore warnings
    warnings.simplefilter("ignore")
    import networkx as nx
    import numpy as np
    import sidis
    rng=sidis.RNG(0)
    import matplotlib.pyplot as plt
    import typing
    from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable, Hashable, List, Any
    from collections import namedtuple

In [ ]:
#export
def ring(N : int = 3,
         left : bool = True,
         right : bool = False,
         loop : bool = False):
    '''
    Return `g`, a ring topology networkx graph with `N` nodes.
    Booleans `left`, `right`, `loop` determine the directed edges.
    '''

    g=nx.MultiDiGraph()

    e=[]

    if left:
        e+=[(i,(i-1)%N) for i in range(N)]
    if right:
        e+=[(i,(i+1)%N) for i in range(N)]
    if loop:
        e+=[(i,i) for i in range(N)]

    g.add_nodes_from([i for i in range(N)])
    g.add_edges_from(e)

    return g

In [ ]:
g=ring(N=3,left=True,right=True,loop=True)

In [ ]:
#exporti
def table(iterable : Iterable, header : Iterable[str]):
    '''
    Creates a simple ASCII table from an iterable and a header.
    Modified from
    https://stackoverflow.com/questions/5909873/how-can-i-pretty-print-ascii-tables-with-python
    '''
    max_len = [len(x) for x in header]
    for row in iterable:
        row = [row] if type(row) not in (list, tuple) else row
        for index, col in enumerate(row):
            if max_len[index] < len(str(col)):
                max_len[index] = len(str(col))

    output = '|' + ''.join([h + ' ' * (l - len(h)) + '|' for h, l in zip(header, max_len)]) + '\n'

    for row in iterable:
        row = [row] if type(row) not in (list, tuple) else row
        output += '|' + ''.join([str(c) + ' ' * (l - len(str(c))) + '|' for c, l in zip(row, max_len)]) + '\n'

    return output


In [ ]:
#export
def print_graph(g : nx.MultiDiGraph,
               string=False):
    '''
    Print the 'node', predecessors', and 'successors' for every node in graph `g`.
    The predecessors are the nodes flowing into a node, 
    and the successors are the nodes flowing out.
    
    Example use:
        g=ring(N=3,left=True,right=True,loop=True)
        print_graph(g)
    '''
    data = [[n, list(g.predecessors(n)), list(g.successors(n))] for n in g.nodes]
    for i in range(len(data)):
        data[i][1]=', '.join([str(i) for i in data[i][1]])
        data[i][2]=', '.join([str(i) for i in data[i][2]])

    header=['Node', 'Predecessors', 'Successors']
    
    if not string:
        print(table(data,header))
    else:
        return table(data,header)


In [ ]:
print_graph(g)

|Node|Predecessors|Successors|
|0   |1, 2, 0     |2, 1, 0   |
|1   |2, 0, 1     |0, 2, 1   |
|2   |0, 1, 2     |1, 0, 2   |



In [ ]:
#export
def parse_kwargs(**kwargs):
    '''
    Evaluate delayed function calls by replacing 
        kwarg=(func,*farg,dict(**fkwarg))
    with
        kwarg=func(*farg,**fkwarg)
     
    Example: kwargs = {a : (np.random.random,1)}
    becomes  kwargs = {a : np.random.random(1)}
    each time this function is called.
    
    Used to randomize kwarg assignment for 
    an exterior function, e.g setting node
    and edge attributes.
    '''
    newkwargs={k:v for k,v in kwargs.items()}
    for k,v in kwargs.items():
        if type(v) is tuple and callable(v[0]):
            if len(v)==1:
                newkwargs[k]=v[0]()
            elif len(v)==2:
                if type(v[-1]) is dict:
                    newkwargs[k]=v[0](**v[1])
                else:
                    newkwargs[k]=v[0](v[1])
            else:
                if type(v[-1]) is dict:
                    newkwargs[k]=v[0](*v[1:-1],**v[-1])
                else:
                    newkwargs[k]=v[0](*v[1:])
    return newkwargs

In [ ]:
parse_kwargs(init=(rng.random,dict(x=0,y=1,shape=8,asint=True)))

{'init': array([1, 0, 0, 0, 1, 1, 1, 1])}

In [ ]:
parse_kwargs(a=(rng.random,dict(x=0,y=1,shape=2))) #specify kwargs explicitly

{'a': array([0.54362499, 0.93507242])}

In [ ]:
parse_kwargs(a = (np.random.uniform,0,1) ) #specify args implicitly

{'a': 0.9957327011121017}

In [ ]:
parse_kwargs(a = (np.random.uniform,0,dict(high=1)) ) #mix the two

{'a': 0.2571581456075053}

In [ ]:
parse_kwargs( a = (np.random.random,) ) #tuple of just func calls default func()

{'a': 0.968339261201048}

In [ ]:
parse_kwargs(b=1,a=np.random.random) #no tuple assigns kwargs normally

{'b': 1, 'a': <function RandomState.random>}

In [ ]:
#export
def give_nodes(g : nx.MultiDiGraph,
               data : Dict[Hashable,dict] = None,
               nodes : Iterable = None,
               **kwargs):
    '''
    Parse and apply any 'kwargs' to a set of 'nodes'.
    If given, 'data' is a dict-of-dicts keyed by node.
    The inner dict is given to the corresponding node.
    '''
    if nodes is None:
        nodes=g.nodes
    
    if kwargs:
        [sidis.give(g.nodes[n],**parse_kwargs(**kwargs)) for n in nodes]
    
    if data:
        for k,v in data.items():
            try:
                g.nodes[k].update(parse_kwargs(**v))
            except KeyError:
                pass


In [ ]:
give_nodes(g,a=(np.random.random,1))
g.nodes(data=True)

NodeDataView({0: {'a': array([0.83597281])}, 1: {'a': array([0.58115496])}, 2: {'a': array([0.00429846])}})

In [ ]:
give_nodes(g,{0:dict(b=1)})
g.nodes(data=True)

NodeDataView({0: {'a': array([0.83597281]), 'b': 1}, 1: {'a': array([0.58115496])}, 2: {'a': array([0.00429846])}})

In [ ]:
give_nodes(g,nodes=[2],c=2)
g.nodes(data=True)

NodeDataView({0: {'a': array([0.83597281]), 'b': 1}, 1: {'a': array([0.58115496])}, 2: {'a': array([0.00429846]), 'c': 2}})

In [ ]:
#exporti
def parse_edges(edges : Union[tuple,List[tuple]], 
                default_key : Hashable = 0
               ):
    '''
    Parse a single edge or list of edges
    into a list of 3-tuples for iterating over 
    a MultiDiGraph, which requires keys. 
    '''
    if type(edges) is tuple:
        edges=[edges]
    if type(edges) is not list:
        edges=list(edges)
    for i in range(len(edges)):
        if len(edges[i])==4: #discard data, last entry
            edges[i]=(edges[i][0],edges[i][1],edges[i][2])
        if len(edges[i])==2: #include key, 3rd entry
            edges[i]=(edges[i][0],edges[i][1],default_key)
    return edges

In [ ]:
#export
def give_edges(g : nx.MultiDiGraph,
               data : Dict[Hashable,dict] = None,
               edges : Iterable = None,
               **kwargs):
    '''
    Parse and apply any 'kwargs' to a set of 'edges'.
    If given, 'data' is a dict-of-dicts keyed by edge.
    The inner dict is given to the corresponding edge.
    '''
    if edges is None:
        edges=g.edges
        
    edges = parse_edges(edges)
    
    if kwargs:
        [sidis.give(g.edges[e],**parse_kwargs(**kwargs)) for e in edges]
    
    if data:
        for k,v in data.items():
            try:
                g.edges[k].update(parse_kwargs(**v))
            except:
                pass

In [ ]:
give_edges(g,d=1)
g.edges(data=True)

OutMultiEdgeDataView([(0, 2, {'d': 1}), (0, 1, {'d': 1}), (0, 0, {'d': 1}), (1, 0, {'d': 1}), (1, 2, {'d': 1}), (1, 1, {'d': 1}), (2, 1, {'d': 1}), (2, 0, {'d': 1}), (2, 2, {'d': 1})])

In [ ]:
give_edges(g,{e:dict(d=i) for i,e in enumerate(g.edges)})
g.edges(data=True)

OutMultiEdgeDataView([(0, 2, {'d': 0}), (0, 1, {'d': 1}), (0, 0, {'d': 2}), (1, 0, {'d': 3}), (1, 2, {'d': 4}), (1, 1, {'d': 5}), (2, 1, {'d': 6}), (2, 0, {'d': 7}), (2, 2, {'d': 8})])

In [ ]:
#export
def node_attrs(g):
    '''
    Unique node data keys.
    '''
    attrs=[]
    for n in g.nodes:
        for attr in g.nodes[n]:
            attrs+=[attr]
    return list(set(attrs))

In [ ]:
node_attrs(g)

['c', 'b', 'a']

In [ ]:
#export
def edge_attrs(g):
    '''
    Unique edge data keys.
    '''
    attrs=[]
    for e in g.edges:
        for attr in g.edges[e]:
            attrs+=[attr]
    return list(set(attrs))

In [ ]:
edge_attrs(g)

['d']

In [ ]:
#export
def node_data(g,*args):
    '''
    Return node attributes 'args' as an array.
    NOTE: The ordering of the array corresponds to the
    ordering of the nodes in the graph.
    '''
    if not args:
        args=node_attrs(g)

    node_data={}

    [sidis.give(node_data,str(arg),
                np.squeeze(np.array([sidis.get(g.nodes[n],arg) for n in g.nodes])))
        for arg in args]

    return node_data

In [ ]:
node_data(g)

{'c': array([None, None, 2], dtype=object),
 'b': array([1, None, None], dtype=object),
 'a': array([0.83597281, 0.58115496, 0.00429846])}

In [ ]:
#export
def edge_data(g,*args):
    '''
    Return edge attributes 'args' as an array.
    NOTE: The ordering of the array corresponds to the
    ordering of the edges in the graph.
    '''
    if not args:
        args=edge_attrs(g)

    edge_data={}

    [sidis.give(edge_data,str(arg), np.array([sidis.get(g.edges[e],arg) for e in g.edges]))
        for arg in args]

    return edge_data

In [ ]:
edge_data(g)

{'d': array([0, 1, 2, 3, 4, 5, 6, 7, 8])}

In [ ]:
#export
def argwhere(*args : List[np.ndarray]):
    '''
    Simplified version of np.argwhere for multiple arrays.
    Returns list of indices where args hold.
    '''
    with warnings.catch_warnings(): #ignore numpy warning
        warnings.simplefilter("ignore")
        if not args:
            return None
        elif len(args)==1:
            return list(np.ravel(np.argwhere(args[0])).astype(int))
        else:
            i=[] #indices
            for arg in args:
                res=list(np.ravel(np.argwhere(arg)).astype(int))
                i+=[res]
            if len(i)==1:
                i=i[0]
            if np.any(i):
                return list(i)

In [ ]:
a=np.array([0,1,2])
A=argwhere(a==0,a==1,a>1,a==10)
A

[[0], [1], [2], []]

In [ ]:
for i in A:
    print(a[i])

[0]
[1]
[2]
[]


In [ ]:
#export
def kwargwhere(g : nx.MultiDiGraph,**kwargs : Dict[str,Any]):
    '''
    Return the node and edges where
    the kwarg equalities hold in the graph.
    '''
    node_k=node_attrs(g)
    edge_k=edge_attrs(g)
    node_i=[]
    edge_i=[]
    for k,v in kwargs.items():
        n_i=[]
        e_i=[]
        if k in node_k:
            for n in g.nodes:
                if g.nodes[n].get(k)==v:
                    n_i+=[n]
            node_i+=[n_i]
        if k in edge_k:
            for e in g.edges:
                if g.edges[e].get(k)==v:
                    e_i+=[e]
            edge_i+=[e_i]

    if len(node_i)==1:
        node_i=node_i[0]
    if len(edge_i)==1:
        edge_i=edge_i[0]
    if node_i and edge_i:
        return node_i,edge_i
    elif node_i:
        return node_i
    elif edge_i:
        return edge_i

In [ ]:
print(g.nodes(data=True))

[(0, {'a': array([0.83597281]), 'b': 1}), (1, {'a': array([0.58115496])}), (2, {'a': array([0.00429846]), 'c': 2})]


In [ ]:
kwargwhere(g,b=1)

[0]

In [ ]:
kwargwhere(g,d=1)

[(0, 1, 0)]

In [ ]:
g.edges[(0,0,0)]['b']=1
print(g.edges(data=True))

[(0, 2, {'d': 0}), (0, 1, {'d': 1}), (0, 0, {'d': 2, 'b': 1}), (1, 0, {'d': 3}), (1, 2, {'d': 4}), (1, 1, {'d': 5}), (2, 1, {'d': 6}), (2, 0, {'d': 7}), (2, 2, {'d': 8})]


In [ ]:
kwargwhere(g,b=1)

([0], [(0, 0, 0)])

In [ ]:
#export
def where(g,*args,**kwargs):
    '''
    Combine the 'argwhere' and 'kwargwhere' functions for the graph.
    '''
    arg_i=argwhere(*args)
    kwarg_i=kwargwhere(g,**kwargs)
    if arg_i and kwarg_i:
        return arg_i,kwarg_i
    elif arg_i:
        return arg_i
    elif kwarg_i:
        return kwarg_i

In [ ]:
where(g,node_data(g)['a']>0.5,b=1)

([0, 1], ([0], [(0, 0, 0)]))

In [ ]:
#exporti
def parse_lengths(g : nx.MultiDiGraph,
                  edges : Union[tuple,List[tuple]],
                  lengths : Union[str,int,List[int]] = 1) -> Union[list,List[list]]:
    '''
    Convert `lengths` corresponding to attributes of each edge into a list of lists.
    `lengths` can be a single integer, an integer for each edge, or a string
    giving the edge attribute holding the length.
    '''
    if type(lengths) is int:
        lengths={e:lengths for e in edges}
    elif type(lengths) is str:
        lengths={e:g.edges[e].get(lengths) for e in edges}
    return lengths

In [ ]:
#export
def convert_edges(g : nx.MultiDiGraph,
                  edges : Union[None,tuple,List[tuple]] = None,
                  lengths : Union[str,int,dict] = 1,
                  node_data : dict = {},
                  label : callable = lambda g,node,iterable : len(g)+iterable,
                  **edge_data
                 ):
    '''
    Converts `edges` in `g` to paths of the given `lengths`. 
    The new paths follow a tree structure, and each new node
    inherits `node_data` and is labeled with `label`.
    The tree structure finds the roots (set of starting nodes)
    in the list of `edges`, and then creates trunks corresponding
    to the paths of maximum length for each node. Then, branches are
    added from the trunk to each of the leaves (terminal nodes), 
    made from new nodes equal to the lengths associated with each path. 
    '''
    
    #default to all edges
    if edges is None:
        edges=g.edges
        
    #parse args
    edges=parse_edges(edges=g.edges(keys=True),default_key=0)
    lengths=parse_lengths(g=g,edges=edges,lengths=lengths)

    #unique first nodes
    roots=set([e[0] for e in edges])
    
    #max path lengths on a per-starting node basis
    trunks={r:max([lengths[e] for e in g.out_edges(r,keys=True) if e in edges]) 
            for r in roots} 
    
    #sort roots by longest trunk length to create largest trunks first
    roots=sorted(roots,
                 key=lambda r: trunks[r],
                 reverse=True) 
    
    #terminal nodes for each branch
    leaves={r:list(g.successors(r)) for r in roots} 

    #now build trunks, then create branches from trunk to edges
    for r in roots:
        trunk=[label(g,node=r,iterable=i) for i in range(trunks[r])]
        if trunk!=[]:
            nx.add_path(g,[r]+trunk,**parse_kwargs(**edge_data))
            give_nodes(g,nodes=trunk,**node_data)
            for edge,length in lengths.items():
                if edge[0]==r: #branch from root
                    
                    if length==trunks[r]: #go to leaf using trunk endpoint
                        branch=[trunk[-1]]+[edge[1]] 
                        
                    else: #create new branch from somewhere in trunk
                        branch=[trunk[length-1]]+[edge[1]]
                        
                    nx.add_path(g,branch,**g.edges[edge]) #apply old edge data
                        
                    give_nodes(g,nodes=branch[:-1],**node_data)
        
    #trim old edges
    for e in edges:
        g.remove_edge(*e)
        

In [ ]:
g=ring()
print_graph(g)
convert_edges(g,lengths=1)
print_graph(g)

|Node|Predecessors|Successors|
|0   |1           |2         |
|1   |2           |0         |
|2   |0           |1         |

|Node|Predecessors|Successors|
|0   |4           |3         |
|1   |5           |4         |
|2   |3           |5         |
|3   |0           |2         |
|4   |1           |0         |
|5   |2           |1         |



In [ ]:
g=nx.MultiDiGraph()
g.add_edges_from([('a','c'),('b','a'),('b','b'),('c','b'),('c','c'),('d','c')])
print_graph(g)
give_edges(g,{e:dict(delay=i) for i,e in enumerate(g.edges)})
print(g.edges(data=True))

|Node|Predecessors|Successors|
|a   |b           |c         |
|c   |a, c, d     |b, c      |
|b   |b, c        |a, b      |
|d   |            |c         |

[('a', 'c', {'delay': 0}), ('c', 'b', {'delay': 1}), ('c', 'c', {'delay': 2}), ('b', 'a', {'delay': 3}), ('b', 'b', {'delay': 4}), ('d', 'c', {'delay': 5})]


In [ ]:
convert_edges(g,
              edges=None,
              lengths='delay',
              node_data={'tau':1},
              label=lambda g,node,iterable : str(node)+'_'+str(iterable+1),
              delay=0
             )
print_graph(g)
print(g.edges(data=True))
print(g.nodes(data=True))

|Node|Predecessors|Successors|
|a   |b_3         |          |
|c   |d_5, c_2    |c_1       |
|b   |b_4, c_1    |b_1       |
|d   |            |d_1       |
|d_1 |d           |d_2       |
|d_2 |d_1         |d_3       |
|d_3 |d_2         |d_4       |
|d_4 |d_3         |d_5       |
|d_5 |d_4         |c         |
|b_1 |b           |b_2       |
|b_2 |b_1         |b_3       |
|b_3 |b_2         |b_4, a    |
|b_4 |b_3         |b         |
|c_1 |c           |c_2, b    |
|c_2 |c_1         |c         |

[('c', 'c_1', {'delay': 0}), ('b', 'b_1', {'delay': 0}), ('d', 'd_1', {'delay': 0}), ('d_1', 'd_2', {'delay': 0}), ('d_2', 'd_3', {'delay': 0}), ('d_3', 'd_4', {'delay': 0}), ('d_4', 'd_5', {'delay': 0}), ('d_5', 'c', {'delay': 5}), ('b_1', 'b_2', {'delay': 0}), ('b_2', 'b_3', {'delay': 0}), ('b_3', 'b_4', {'delay': 0}), ('b_3', 'a', {'delay': 3}), ('b_4', 'b', {'delay': 4}), ('c_1', 'c_2', {'delay': 0}), ('c_1', 'b', {'delay': 1}), ('c_2', 'c', {'delay': 2})]
[('a', {}), ('c', {}), ('b', {}), ('d'

In [ ]:
#export
def relabel_graph(g : nx.MultiDiGraph,
            mapping : Union[None,callable,dict] = None):
    '''
    Relabel nodes in place with desired 'mapping', and store the 
    `mapping` and `inverse_mapping` as attributes of `g`. 
    Can be called again without args to relabel to the original map,
    which switches the `mapping` and `inverse_mapping`.
    If `mapping` is None and `g` has no `mapping`, 
    defaults to replacing nodes with integers.
    If `mapping` is None and `g` has a `mapping`, uses that.
    Otherwise, `mapping` is a callable or dict keyed with old node labels 
    as keys and new node labels as values.
    '''
    if mapping is None:
        if not g.__dict__.get('mapping'):
            mapping={n:i for i,n in enumerate(g.nodes)}
        else:
            mapping=g.mapping
            
    elif callable(mapping):
        mapping=mapping(g)
    
    inverse_mapping={v:k for k,v in mapping.items()}
    def relabel_nodes(G, mapping):
        H = nx.MultiDiGraph()
        H.add_nodes_from(mapping.get(n, n) for n in G)
        H._node.update((mapping.get(n, n), d.copy()) for n, d in G.nodes.items())
        if G.is_multigraph():
            new_edges = [
                (mapping.get(n1, n1), mapping.get(n2, n2), k, d.copy())
                for (n1, n2, k, d) in G.edges(keys=True, data=True)
            ]

            # check for conflicting edge-keys
            undirected = not G.is_directed()
            seen_edges = set()
            for i, (source, target, key, data) in enumerate(new_edges):
                while (source, target, key) in seen_edges:
                    if not isinstance(key, (int, float)):
                        key = 0
                    key += 1
                seen_edges.add((source, target, key))
                if undirected:
                    seen_edges.add((target, source, key))
                new_edges[i] = (source, target, key, data)

            H.add_edges_from(new_edges)
        else:
            H.add_edges_from(
                (mapping.get(n1, n1), mapping.get(n2, n2), d.copy())
                for (n1, n2, d) in G.edges(data=True)
            )
        H.graph.update(G.graph)
        return H
    gnew=relabel_nodes(g,mapping)
    g.__dict__.update(gnew.__dict__)
    g.mapping=inverse_mapping
    g.inverse_mapping=mapping

In [ ]:
relabel_graph(g)
print_graph(g)

|Node|Predecessors|Successors|
|0   |11          |          |
|1   |8, 14       |13        |
|2   |12, 13      |9         |
|3   |            |4         |
|4   |3           |5         |
|5   |4           |6         |
|6   |5           |7         |
|7   |6           |8         |
|8   |7           |1         |
|9   |2           |10        |
|10  |9           |11        |
|11  |10          |12, 0     |
|12  |11          |2         |
|13  |1           |14, 2     |
|14  |13          |1         |



In [ ]:
relabel_graph(g)
print_graph(g)

|Node|Predecessors|Successors|
|a   |b_3         |          |
|c   |d_5, c_2    |c_1       |
|b   |b_4, c_1    |b_1       |
|d   |            |d_1       |
|d_1 |d           |d_2       |
|d_2 |d_1         |d_3       |
|d_3 |d_2         |d_4       |
|d_4 |d_3         |d_5       |
|d_5 |d_4         |c         |
|b_1 |b           |b_2       |
|b_2 |b_1         |b_3       |
|b_3 |b_2         |b_4, a    |
|b_4 |b_3         |b         |
|c_1 |c           |c_2, b    |
|c_2 |c_1         |c         |



In [ ]:
#export
def sort_graph(g : nx.MultiDiGraph,
               nodes_by='in_degree', #g.in_degree, #sorting this function over nodes
               node_key=lambda t:sidis.get(t,-1,-1), #last element of sorting tuple
               node_args=(), #not accessing any attributes by default
               nodes_ascending=True,
               edges_by=None, #not generating function evals to sort
               edge_key=None,#orders edges, defaults to linear comb of node sort
               edge_args=(), #not accessing any edge attrs by default
               edges_ascending=False,
               relabel=False #relabel to integers
              ) -> None:
    '''
    Sort the graph in place by changing node and edge order.
    See `sidis.sort` documentation for explanation of by, key, and args.
    Default behavior is to sort nodes by in-degree, and edges by increasing node label,
    after relabling nodes to integers. Stores result in 'sorting' attribute.
    '''
    #parse args; get node sorting attr if str
    if type(nodes_by) is str:
        nodes_by=sidis.get(g,nodes_by)
    #if no edge key given default to ordering by linear comb of node func
    if edge_key is None:
        edge_key=lambda t:100*nodes_by(t[0])-10*nodes_by(t[1])
    
    #sort nodes
    node_sorting=sidis.sort(g.nodes,
                            *node_args,
                            by=nodes_by,
                            key=node_key,
                            reverse=nodes_ascending)

    #sort returns tuples of (node,nodes_by(node)), so extract nodes and data
    if nodes_by is None:
        nodes=[(n,g.nodes[n]) for n in node_sorting]
    else:
        nodes=[(n[0],g.nodes[n[0]]) for n in node_sorting]  
    
    #sort edges
    edge_sorting=sidis.sort(list(g.edges(keys=True)),
                            *edge_args,
                            by=edges_by,
                            key=edge_key,
                            reverse=edges_ascending)
    
    #extract edge,data tuple
    if edges_by is None:
        edges=[(*e,g.edges[e]) for e in edge_sorting]
    else:
        edges=[(*e[0],g.edges[e[0]]) for e in edge_sorting]        
    
    #wipe graph and add new nodes/edges in order
    g.clear()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    
    #relabel to new ranking if desired
    if relabel:
        mapping={n:i for i,n in enumerate([node[0] for node in nodes])}
        relabel_graph(g,mapping)
        new_node_sorting=[]
        for node,rank in node_sorting:
            new_node_sorting+=[(g.inverse_mapping[node],rank)]
        node_sorting=new_node_sorting
    
    sorting=nx.utils.groups(dict(node_sorting))
    g.sorting={k:list(v) for k,v in sorting.items()}             

In [ ]:
g=nx.MultiDiGraph()
g.add_edges_from([('a','c'),('b','a'),('b','b'),('c','b'),('c','c'),('d','c')])
print('Before')
print_graph(g)
print('After sorting and relabeling')
sort_graph(g,relabel=True)
print_graph(g)
print('Sorting result: g.sorting')
g.sorting

Before
|Node|Predecessors|Successors|
|a   |b           |c         |
|c   |a, c, d     |b, c      |
|b   |b, c        |a, b      |
|d   |            |c         |

After sorting and relabeling
|Node|Predecessors|Successors|
|0   |0, 2, 3     |0, 1      |
|1   |0, 1        |1, 2      |
|2   |1           |0         |
|3   |            |0         |

Sorting result: g.sorting


{3: [0], 2: [1], 1: [2], 0: [3]}

In [ ]:
notebook2script()

Converted 00_graph_functions.ipynb.
Converted 01_model_functions.ipynb.
Converted 02_network_class.ipynb.
No export destination, ignored:
#export
import warnings
with warnings.catch_warnings(): #ignore warnings
    warnings.simplefilter("ignore")
    import networkx as nx
    import numpy as np
    import sidis
    rng=sidis.RNG(0)
    import matplotlib.pyplot as plt
    import typing
    from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable, Hashable, List, Any
    from collections import namedtuple
    import einops
    import numba
    from numba import njit
    from scipy import stats
    import scipy.optimize as sciopt
    
    from networkm.graph_functions import *
    from networkm.model_functions import *
    #from networkm.network_class import *
No export destination, ignored:
#export
@njit
def query_rise(iterator_matrix,
               time_delay_matrix,
               sigmoid_constant_matrix,
               time_constant_matrix,
             

AttributeError: 'NoneType' object has no attribute 'start'